In [1]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, Regex
import tokenizers
import pandas as pd

In [2]:
df = pd.read_csv('/home/web/files_for_research_Vova/processed_data.csv',\
                 usecols=['review_translate',
                                                            'dataset_name',
                                                            'rating',
                                                           'translated'])

In [3]:
tokenizer = Tokenizer(models.BPE.from_file(vocab='/home/web/files_for_research_Vova/tokenizer_30k.json',
            merges='/home/web/files_for_research_Vova/merges_tokenizer.txt',
                                          end_of_word_suffix='</w>'))
tokenizer.pre_tokenizer = pre_tokenizers.Split(Regex(r"[\w'-]+|[^\w\s'-]+"),'removed', True)

In [4]:
df['review_translate'] = df['review_translate'].str.lower()

In [5]:
df['encoded'] = tokenizer.encode_batch(df['review_translate'].values)

In [6]:
df['encoded'] = df['encoded'].apply(lambda x: x.tokens)

# Make word2vec model

In [7]:
import gensim
import time
def save_vectors(model, name):
    model.wv.save_word2vec_format(name, binary=True)

In [8]:
def train(type_model, data, path, epochs=10):
    start = time.time()
    model = type_model(size=300, window=10,
                                               min_count=10, workers=30, sg=0, hs=1,
                                               cbow_mean=1,
                                               sample=1e-3,
                                               alpha=0.001)
    model.build_vocab(data, progress_per=100_000)
    model.train(data, epochs=epochs, report_delay=1, total_examples=model.corpus_count)
    end = time.time()
    save_vectors(model, path)
    print(f'It took : {(end-start)/60} minutes')

In [9]:
train(gensim.models.Word2Vec, \
      df['encoded'].values.tolist(), '/home/web/files_for_research_Vova/embeddings_w2v.bin')

It took : 7.764649343490601 minutes


In [10]:
embeddings = gensim.models.KeyedVectors.load_word2vec_format('/home/web/files_for_research_Vova/embeddings_w2v.bin', binary=True)

In [11]:
len(embeddings.wv.vocab)

/tmp/ipykernel_2956820/1721352991.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  len(embeddings.wv.vocab)


29602

In [12]:
df['encoded'].values[0]

['якісна</w>',
 'пласт',
 'маса</w>',
 ',</w>',
 'пере',
 'воро',
 'т</w>',
 '360</w>',
 'граду',
 'св',
 'в</w>']